# Algorithme de Grover

Problème de recherche, exemple:
- identifier une entrée dans une dase de données non-structurée
- identifier la solution d'un problème NP-complexe


Solutions possibles $x\in\{0,1\}^n$ (encodées sur $n$ bits)

On dispose d'une fonction $f$ permettant d'identifier la solution si celle-ci est donnée:

- $f(x) = 1$ si $x$ est solution
- $f(x) = 0$ sinon

On appelle une telle fonction un "oracle"

On se sert de l'oracle pour construire une porte quantique que l'on peut insérer dans un circuit:

\begin{equation*}
    U_f \ket{x}\ket{-} = (-1)^{f(x)} \ket{x}\ket{-}
\end{equation*}

Cette porte logique "reconnaît" la solution du problème décrit par $f$ et a pour effet d'inverser le signe de la composante vectorielle dans la direction de la solution.

\begin{equation*}
    U_f \ket{x}\ket{y} = \ket{x}\ket{y \oplus f(x)}
\end{equation*}

Dans le cas de l'oracle de Grover, seulement un qubit additionnel ('ancilla') est nécessaire pour mémoriser le résultat $f(x)\in\{0,1\}$. En initialisant le qubit $\ket y$ avec l'état $\ket- = \frac{1}{\sqrt 2}(\ket 0 - \ket 1)$, l'action de l'oracle se réduit à un changement de phase:

\begin{equation*}
    U_f \ket{x}\ket{-} = (-1)^{f(x)} \ket{x}\ket{-}
\end{equation*}

**Est-ce que l'on peut toujours faire cela? Utiliser $\ket{---}$ et toujours réduire à une phase? --> I believe so, the ancilla represents a practical reality, but can be removed from the theoretical picture**

![title](../figures/grover_full.png)

![title](../figures/grover_iter.png)